# Init

## install

In [ ]:
# !pip install matplotlib
# !pip install moviepy

## Imports

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import moviepy.editor as mp

## Functions

In [ ]:
def search_file(directory, filename):
    """
    This function searches for a file with a given filename within a specified directory and its subdirectories.

    :param directory: The directory in which to search for the file.
    :type directory: str
    :param filename: The name of the file to search for.
    :type filename: str
    :return: The full path of the file if found, otherwise returns the string "Not found".
    :rtype: str
    """
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

def gif_to_mp4(path):
    """
    Converts a GIF file to an MP4 file.

    This function takes the path of a GIF file as input, converts it to an MP4 file, and saves the resulting MP4 file in the same directory as the input GIF file. The name of the output MP4 file is the same as the input GIF file, with the file extension changed from `.gif` to `.mp4`.

    Args:
        path (str): The path of the input GIF file.

    Returns:
        None
    """
    clip = mp.VideoFileClip(path)
    save_path = path.split(".")[0]+".mp4"
    clip.write_videofile(save_path)

## Classes

In [ ]:
class Binary_loader:
    """
    A class for loading binary data and converting it into an animation.

    This class provides methods for loading binary data from a file and converting a sequence of binary frames into an animated GIF. The `load_binary` method loads binary data from a specified file and returns it as a NumPy array. The `binary_frames_to_animation` method takes a sequence of binary frames and converts them into an animated GIF, which is saved to the specified directory.

    Attributes:
        None
    """
    def load_binary(self, data_path, n_frames_to_be_acquired, fname="data.bin", image_x_size=512, image_y_size=512):
        """
        Loads binary data from a file.

        This method takes the path of a binary data file as input, along with the number of frames to be acquired and the dimensions of each frame. It loads the binary data from the specified file and returns it as a NumPy array.

        Args:
            data_path (str): The path of the binary data file.
            n_frames_to_be_acquired (int): The number of frames to be acquired from the binary data file.
            fname (str): The name of the binary data file. Defaults to "data.bin".
            image_x_size (int): The width of each frame in pixels. Defaults to 512.
            image_y_size (int): The height of each frame in pixels. Defaults to 512.

        Returns:
            np.ndarray: A NumPy array containing the loaded binary data.
        """
        # load binary file from suite2p_folder from unit
        image_size=image_x_size*image_y_size
        fpath = search_file(data_path, fname)
        binary = np.memmap(fpath,
                            dtype='uint16',
                            mode='r',
                            shape=(n_frames_to_be_acquired, image_x_size, image_y_size))
        return binary
    
    def binary_frames_to_animation(self, frames, frame_range=[0, -1], save_dir="animation"):
        """
        Converts a sequence of binary frames into an animated GIF.

        This method takes a sequence of binary frames as input, along with the range of frames to include in the animation and the directory in which to save the resulting GIF. It converts the specified frames into an animated GIF and saves it to the specified directory.

        Args:
            frames (np.ndarray): A NumPy array containing the sequence of binary frames.
            frame_range (List[int]): A list specifying the range of frames to include in the animation. Defaults to [0, -1], which includes all frames.
            save_dir (str): The dir.VideoFileClip(pectory in which to save the resulting GIF. Defaults to "animation".

        Returns:
            animation.ArtistAnimation: An instance of `animation.ArtistAnimation` representing the created animation.
        """
        import matplotlib.animation as animation

        range_start, range_end = frame_range
        gif_save_path = os.path.join(save_dir, f"{range_start}-{range_end}.gif")

        images = []
        fig = plt.figure(figsize=(10, 5))
        ax = fig.add_subplot(111)
        for i, frame in enumerate(frames):
            if i%100 == 0:
                print(i)
            p1 = ax.text(512/2-50, 0, f"Frame {i}", animated=True)
            p2 = ax.imshow(frame, animated=True)
            images.append([p1, p2])
            if i > range_end:
                break
        ani = animation.ArtistAnimation(fig, images, interval=50, blit=True,
                                        repeat_delay=1000)
        ani.save(gif_save_path)
        return ani

# Main

## Settings

In [ ]:
# Set the path to the binary data
data_path = r"D:\DON-018975\20240304\Red_alignment"

# Set the frame ranges to be converted into animations
frame_ranges = [[50, 100], [500, 1000], [5000, 6000], [0, 9001]]

# Set the dimensions of each frame in pixels
image_x_size, image_y_size = 512, 512

# Set the number of frames to be acquired from the binary data file
n_frames_to_be_acquired = 111480 # sum of frames of every mesc file used for binary

## Run Binary to mp4 (save location: *data_path/animation*)

In [ ]:
# Create an instance of the Binary_loader class
loader = Binary_loader()

# Load binary data from the specified file using the load_binary method of the Binary_loader instance
frames = loader.load_binary(data_path, n_frames_to_be_acquired, "data.bin", image_x_size, image_y_size)

# Set the path for saving the animations
animation_path = os.path.join(data_path, "animation")
if not os.path.exists(animation_path):
    os.mkdir(animation_path)

# Loop through the specified frame ranges
for start_frame, end_frame in frame_ranges:
    # Convert the specified range of binary frames into an animated GIF using the binary_frames_to_animation method of the Binary_loader instance
    loader.binary_frames_to_animation(frames=frames, frame_range=[start_frame, end_frame], save_dir=animation_path)
    # Convert the resulting GIF into an MP4 file using the gif_to_mp4 function
    gif_to_mp4(os.path.join(animation_path, f"{start_frame}-{end_frame}")+".gif")
